In [ ]:
import tensorflow as tf
import numpy as np
import shutil

In [ ]:
shutil.rmtree("outdir", ignore_errors = True) # start fresh each time

In [ ]:
def decode_csv(row):
    cols = tf.decode_csv(row, record_defaults=[[0],['house'],[0]])
    features = {'sq_footage': cols[0], 'type': cols[1]}
    price = cols[2] # "label"
    return features, price

dataset = tf.data.Dataset.list_files("property*")            \
                         .flat_map(tf.data.TextLineDataset)  \
                         .map(decode_csv)
        
dataset = dataset.shuffle(1000) \
                 .repeat()  \
                 .batch(3)
        
featcols = [
    tf.feature_column.numeric_column("sq_footage"),
    tf.feature_column.indicator_column(
        tf.feature_column.categorical_column_with_vocabulary_list("type", ["house", "apt"])
    )
]
        
def train_input_fn():
    return dataset.make_one_shot_iterator().get_next()

def serving_input_fn():
    json = { 'sq_footage' : tf.placeholder(tf.int32, [None]),
             'prop_type'  : tf.placeholder(tf.string, [None])
           }
        
    features = { 'sq_footage': json['sq_footage'],
                 'type': json['prop_type']
               }
    return tf.estimator.export.ServingInputReceiver(features, json)

run_config = tf.estimator.RunConfig(model_dir="outdir",
                                    save_summary_steps=10,
                                    save_checkpoints_steps=1000)
        
export_last = tf.estimator.LatestExporter("houseprice",
                                          serving_input_fn)
        
train_spec = tf.estimator.TrainSpec(input_fn=train_input_fn, max_steps=5000)
        
eval_spec = tf.estimator.EvalSpec(input_fn=train_input_fn, ## use real evaldata !
                                  steps=10, # evals on 100 batches
                                  throttle_secs=3, # eval no more than every 3 sec
                                  exporters=export_last)

#model = tf.estimator.LinearRegressor(featcols, config=run_config)
model = tf.estimator.DNNRegressor([3, 2, 2], featcols, config=run_config)

tf.estimator.train_and_evaluate(model, train_spec, eval_spec)

In [ ]:
# try to read the Dataset by hand
features, labels = train_input_fn()
with tf.Session() as sess:
    for i in range(40):
        feat, lab = sess.run([features, labels])
        print("feat:" + str(feat) + " label:" + str(lab))